### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [3]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [4]:
X_test[['Rooms', 'Square', 'LifeSquare', 'MedPriceByDistrict', 'MedPriceByFloorYear']].head(3)

,Rooms,Square,LifeSquare,MedPriceByDistrict,MedPriceByFloorYear
0,1.0,43.091028,44.337653,119817.448457,165526.307915
1,1.0,37.753972,21.042126,177820.823017,181467.964769
2,2.0,44.581728,31.525382,338569.019198,189145.451091


In [5]:
import urllib.request
import json      

def get_prediction(Rooms, Square, LifeSquare, MedPriceByDistrict, MedPriceByFloorYear):
    body = {'Rooms': Rooms, 
                            'Square': Square,
                            'LifeSquare': LifeSquare, 
                            'MedPriceByDistrict': MedPriceByDistrict,
                            'MedPriceByFloorYear': MedPriceByFloorYear} 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [ ]:
get_prediction("1.0", "36.847630", "19.094182","171145.169069","188642.277553")

In [7]:
%%time
predictions = X_test[['Rooms', 'Square', 'LifeSquare', 'MedPriceByDistrict', 'MedPriceByFloorYear']].iloc[:500].apply(lambda x: get_prediction(x[0], 
                                                                                               x[1], 
                                                                                               x[2], 
                                                                                               x[3], 
                                                                                               x[4]), 1)

CPU times: user 340 ms, sys: 55.7 ms, total: 396 ms
Wall time: 5.16 s


In [8]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.9803161348046525

In [7]:
#X_test[['Rooms', 'Square', 'LifeSquare', 'MedPriceByDistrict', 'MedPriceByFloorYear']].head(3)

Давайте подумаем, где наше api может сломаться

In [9]:
get_prediction("1.0", "36.847630", "19.094182","171145.169069","188642.277553")

0.13518841848029955